<a href="https://colab.research.google.com/github/AlaEmrani/Daily_challenges/blob/master/up_down.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q PyDrive

# HERE YOUR FILE ID ( GET IT WITH THE SHARING URL: https://drive.google.com/open?id=1Soh3zXLXt2lT7b_3FcWWyeOCC7SnOxK0 )
chr1_id= '1WBPkjNDksTkOFQmdM_xRCLn8fgTdLNBw'
chr2_id= '1DCLQaqoTo1PmyhERdn55GVhbaBh7BOIE'

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import zipfile, os

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

if not os.path.exists('Data'):
    os.makedirs('Data')
    
# DOWNLOAD ZIP
print ("Downloading zip file")
chr_zip = drive.CreateFile({'id': chr1_id})
chr_zip.GetContentFile('chr_1.zip')

chr_zip = drive.CreateFile({'id': chr2_id})
chr_zip.GetContentFile('chr_2.zip')

# UNZIP ZIP
print ("Uncompressing zip file")
zip_ref = zipfile.ZipFile('chr_1.zip', 'r')
zip_ref.extractall('Data/')
zip_ref.close()

zip_ref = zipfile.ZipFile('chr_2.zip', 'r')
zip_ref.extractall('Data/')
zip_ref.close()

Uncompressing zip file


In [2]:
%cd Data
os.makedirs('images')
!ls

/content/Data
chromosomes  chromosomes_series2  images  __MACOSX


In [3]:
import numpy as np
import random
import cv2
import image
import matplotlib.pyplot as plt
import glob
import os

In [4]:
for root, dirs, files in os.walk('chromosomes'):
    for name in files:
        if name.endswith('.jpg'):
          new_name = root.split('/')[1] + '-' + os.path.splitext(name)[0] + os.path.splitext(name)[1]
          os.rename('chromosomes/' + root.split('/')[1] +'/'+ name, 'chromosomes/' + root.split('/')[1] +'/'+ new_name)
          print('----'+ new_name)
#           print(name)

Streaming output truncated to the last 5000 lines.
----288_15-chr-12-0.jpg
----288_15-chr-21-0.jpg
----288_15-chr-1-0.jpg
----288_15-chr-19-1.jpg
----288_15-chr-5-0.jpg
----288_15-chr-6-1.jpg
----288_15-chr-2-0.jpg
----288_15-chr-16-0.jpg
----288_15-chr-10-1.jpg
----288_15-chr-14-1.jpg
----288_15-chr-15-0.jpg
----288_15-chr-17-0.jpg
----288_15-chr-8-1.jpg
----288_15-chr-11-1.jpg
----288_15-chr-9-0.jpg
----288_15-chr-13-1.jpg
----288_15-chr-19-0.jpg
----288_15-chr-18-1.jpg
----288_15-chr-6-0.jpg
----288_15-chr-21-1.jpg
----288_15-chr-10-0.jpg
----288_15-chr-18-0.jpg
----288_15-chr-7-0.jpg
----288_15-chr-20-1.jpg
----288_15-chr-4-1.jpg
----288_15-chr-1-1.jpg
----288_15-chr-11-0.jpg
----288_15-chr-3-0.jpg
----288_15-chr-7-1.jpg
----288_15-chr-15-1.jpg
----288_15-chr-5-1.jpg
----288_15-chr-22-0.jpg
----288_15-chr-12-1.jpg
----288_15-chr-9-1.jpg
----288_15-chr-20-0.jpg
----288_15-chr-3-1.jpg
----288_15-chr-17-1.jpg
----288_15-chr-22-1.jpg
----288_15-chr-23-1.jpg
----288_15-chr-23-0.jpg
----

In [6]:
import shutil
import fnmatch

def gen_find(filepat,top):
    for path, dirlist, filelist in os.walk(top):
      for name in fnmatch.filter(filelist,filepat):
          yield os.path.join(path,name)
            
src = 'chromosomes' # input
dst = 'images' # desired     

filesToMove = gen_find('*.jpg',src)
for name in filesToMove:
    shutil.move(name, dst)

In [7]:
### removing bad samples
images_names = []
processed_names = []
l = []
images_names = os.listdir('images')
processed_names = [os.path.splitext(i)[0] for i in images_names]
l = [i[:i.find('c')-1] for i in processed_names]

list1 = []
for i in np.unique(l):
  list1.append([k for k in processed_names if k.startswith(i)])
  
new_list = []
bad_form = []
for sample in list1:
  name = sample[0]
  for j in [x for x in range(25) if x != 21]:
    patt = name[:name.find('r')+2] + str(j) + '-'
    if len([k for k in sample if k.startswith(patt)]) > 2:
      bad_form.append(sample)
      break

new_list = [x for x in list1 if x not in bad_form]
all_imgs = [item+'.jpg' for sublist in new_list for item in sublist]


In [9]:
!ls

chromosomes  chromosomes_series2  images  __MACOSX


In [10]:
### add padding

def add_padding(im):
  old_size = im.shape[:2] # old_size is in (height, width) format
  if (old_size[0]>128 or old_size[1]>32):
      scaling = max(old_size[0]/128 , old_size[1]/32)
      new_size_1 = int(round(old_size[1] // scaling))
      new_size_0 = int(round(old_size[0] // scaling))
      im = cv2.resize(im, (new_size_1, new_size_0))
      delta_w = 32  - int(round(new_size_1))
      delta_h = 128 - int(round(new_size_0))
      top, bottom = int(delta_h//2), int(delta_h-(delta_h//2))
      left, right = int(delta_w//2), int(delta_w-(delta_w//2))

      color = [0, 0, 0]
#       print(left, 'aaaa')
      new_im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT,
                              value=color)
#       plt.imshow(new_im)
#       cv2.imwrite('chromosomes_resized/' + name_of_file_train[i-1]+'/' + index_train[k], new_im)
#       image0_train.append(new_im)
#       image0_train.append(np.flipud(new_im))
  else:
      delta_w = 32  - old_size[1]
      delta_h = 128 - old_size[0]
      top, bottom = delta_h//2, delta_h-(delta_h//2)
      left, right = delta_w//2, delta_w-(delta_w//2)
      
      color = [0, 0, 0]
#       print(right, left, 'bbbb')
      new_im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT,
                              value=color)
#       plt.imshow(new_im)
#       cv2.imwrite('chromosomes_resized/' + name_of_file_train[i-1]+'/' + index_train[k], new_im)
#       image0_train.append(new_im)
#       image0_train.append(np.flipud(new_im))
  return new_im

In [ ]:
#create trainin data

Data_dir = 'images/'
training_data = []
IMG_SIZE_1, IMG_SIZE_2 = 32, 128
random.seed(10)


def create_training_data():
  for img in all_imgs:
    img_array = cv2.imread(os.path.join(Data_dir,img), cv2.IMREAD_GRAYSCALE)
    ### flip images randomly
    if random.randint(0, 1) == 1:
      new_array = add_padding(np.flip(img_array))
      training_data.append([new_array, 1])
    else:
      new_array = add_padding(img_array)
      training_data.append([new_array, 0])

create_training_data() 
random.shuffle(training_data)


In [13]:
print(np.eye(2)[0])

[1. 0.]


In [ ]:
### create train and label dataset

X = []
y = []

for features, labels in training_data:
  X.append(features)
  y.append(labels)


X = np.array(X).reshape(-1, IMG_SIZE_2, IMG_SIZE_1, 1)
X = X/255.0

y = np.array(y).reshape(-1,1)

In [ ]:
import keras
from sklearn.model_selection import train_test_split


X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
# X_train = X_train/255.0
# X_valid = X_valid/255.5

In [ ]:
######################################   defining the model

In [ ]:
import tensorflow as tf
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import optimizers


In [ ]:

model = Sequential()

model.add(Conv2D(256, (3, 3), input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (2, 2)))
model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(256, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

# model.add(Dense(256))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))

model.add(Dense(128))
model.add(Activation('relu'))


model.add(Dense(1))
model.add(Activation('sigmoid'))

# sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

adm = optimizers.adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=1e-6, amsgrad=False)
model.compile(loss='binary_crossentropy',
              optimizer= adm,
              metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=32, epochs=8, validation_data=(X_valid, y_valid))

Train on 129453 samples, validate on 32364 samples
Epoch 1/8
129453/129453 [==============================] - 142s 1ms/step - loss: 0.3471 - acc: 0.8356 - val_loss: 0.2806 - val_acc: 0.8768
Epoch 2/8
129453/129453 [==============================] - 140s 1ms/step - loss: 0.2510 - acc: 0.8922 - val_loss: 0.2293 - val_acc: 0.9033
Epoch 3/8
129453/129453 [==============================] - 140s 1ms/step - loss: 0.2147 - acc: 0.9117 - val_loss: 0.2124 - val_acc: 0.9133
Epoch 4/8
129453/129453 [==============================] - 140s 1ms/step - loss: 0.1961 - acc: 0.9208 - val_loss: 0.2287 - val_acc: 0.9067
Epoch 5/8
129453/129453 [==============================] - 140s 1ms/step - loss: 0.1803 - acc: 0.9283 - val_loss: 0.1941 - val_acc: 0.9216
Epoch 6/8
129453/129453 [==============================] - 141s 1ms/step - loss: 0.1682 - acc: 0.9336 - val_loss: 0.1903 - val_acc: 0.9250
Epoch 7/8
129453/129453 [==============================] - 140s 1ms/step - loss: 0.1573 - acc: 0.9382 - val_loss: 0

In [ ]:
########################################################################

In [ ]:
model_json = model.to_json()
with open("orientation_model.json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
model.save_weights("orientation_model.h5")
print("Saved model to disk") 


Saved model to disk


In [ ]:
from keras.models import model_from_json


json_file = open('orientation_model.json','r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
#load woeights into new model
loaded_model.load_weights("orientation_model.h5")
print("Loaded Model from disk")

#compile and evaluate loaded model
# adm = optimizers.adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=1e-6, amsgrad=False)
loaded_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
loss,accuracy = model.evaluate(X_valid,y_valid)
print('loss:', loss)
print('accuracy:', accuracy)


Loaded Model from disk
32364/32364 [==============================] - 11s 326us/step
loss: 0.1950941190872668
accuracy: 0.9234025460388086


In [ ]:
a = cv2.imread('images/101_01-chr-18-1.jpg', cv2.IMREAD_GRAYSCALE)
b = cv2.imread('images/401_01-chr-8-1.jpg', cv2.IMREAD_GRAYSCALE)
x = []
x.append(np.flip(add_padding(a)))
x.append(add_padding(b))

In [ ]:
x = np.array(x).reshape(-1, 128, 32, 1)
x = x/255.0
x.shape

(2, 128, 32, 1)

In [ ]:
print((np.argmax(model.predict(y), 1)))


[0]


In [ ]:
a = [1 if i[0] > .5 else 0 for i in model.predict(x)]

In [ ]:
from google.colab import files
files.download('orientation_model.json') 
files.download('orientation_model.h5') 

chromosomes	     images    orientation_model.h5    up-down_model.h5
chromosomes_series2  __MACOSX  orientation_model.json  up-down_model.json


In [ ]:
if model.predict(y) > .5:
  print('ala')

ala
